# EVAC

In [7]:
import numpy as np
from scipy.ndimage import label
from dipy.nn.evac import EVACPlus
from dipy.io.image import load_nifti, save_nifti
from dipy.viz.horizon.app import horizon
import tensorflow as tf

In [11]:
file_path = '/media/juan2634/ROCKET-NANO/MRI_Analysis/ELP/BIDS_ELP/sub-P007/anat/sub-P007_run-1_T1w.nii.gz'

In [12]:
data, affine, img, voxsize = load_nifti(file_path, return_img=True, return_voxsize=True)
evac = EVACPlus()
mask_volume = evac.predict(data, affine, voxsize)

fetched /home/juan2634/.dipy/evac/evac_default_weights.h5
1/1 [==============================] - 2s 2s/step


In [4]:
print(mask.shape)
print(np.mean(mask))

NameError: name 'mask' is not defined

In [ ]:
horizon(images=[(mask*image, affine)])

Original shape: (256, 256, 150)
Resized to RAS shape: (192, 285, 223)


In [13]:
chunks, n_chunk = label(mask_volume)
u, c = np.unique(chunks[chunks != 0], return_counts=True)
target_idx = u[np.argmax(c)]
new_mask = np.where(chunks==target_idx, 1, 0)
save_nifti('/media/juan2634/ROCKET-NANO/MRI_Analysis/ELP/BIDS_ELP/derivatives/DWI/DIPY/sub-P007/input_synb0/T1.nii.gz', (new_mask*data).astype(np.float32), affine)

In [ ]:
horizon(images=[(new_mask*image, affine)])

Original shape: (256, 256, 150)
Resized to RAS shape: (192, 285, 223)


# Median otsu

In [5]:
import numpy as np
from dipy.data import get_fnames
from dipy.io.image import load_nifti
from dipy.segment.mask import median_otsu
from dipy.viz.horizon.app import horizon

In [6]:
data_fnames = get_fnames('scil_b0')
data, affine = load_nifti(data_fnames[1])
data = np.squeeze(data)
print(data.shape)

(128, 128, 65)


In [7]:
b0_mask, mask = median_otsu(data, median_radius=2, numpass=1)

In [8]:
horizon(images=[(data.astype(np.float32), affine)])

Original shape: (128, 128, 65)
Resized to RAS shape: (128, 128, 65)


# Tissue Segmentation

In [ ]:
import numpy as np
import dipy.reconst.shm as shm
import dipy.direction.peaks as dp

from dipy.denoise.localpca import mppca
from dipy.core.gradients import gradient_table, unique_bvals_tolerance
from dipy.io.gradients import read_bvals_bvecs
from dipy.viz.horizon.app import horizon
from dipy.io.image import load_nifti
from dipy.segment.mask import median_otsu
from dipy.segment.tissue import TissueClassifierHMRF

from dipy.data import get_sphere, get_fnames
sphere = get_sphere('symmetric724')

In [ ]:
fraw, fbval, fbvec, t1_fname = get_fnames('cfin_multib')

data, affine = load_nifti(fraw)
bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
gtab = gradient_table(bvals, bvecs)

In [ ]:
bvals = gtab.bvals
bvecs = gtab.bvecs

sel_b = np.logical_or(np.logical_or(bvals == 0, bvals == 1000), bvals == 2000)
data = data[..., sel_b]

In [ ]:
gtab = gradient_table(bvals[sel_b], bvecs[sel_b])

In [ ]:
b0_mask, mask = median_otsu(data, median_radius=2, numpass=1, vol_idx=[0, 1])

print(data.shape)

(96, 96, 19, 67)


In [ ]:
denoised_arr = mppca(data, mask=mask, patch_radius=2)

c:\Users\bpmju\anaconda3\lib\site-packages\dipy\denoise\localpca.py:285: RuntimeWarning: invalid value encountered in true_divide
  denoised_arr = thetax / theta


In [ ]:
qball_model = shm.QballModel(gtab, 8)

In [ ]:
peaks = dp.peaks_from_model(model=qball_model, data=denoised_arr,
                            relative_peak_threshold=.5,
                            min_separation_angle=25,
                            sphere=sphere, mask=mask)

ap = shm.anisotropic_power(peaks.shm_coeff)

In [ ]:
horizon(images=[(ap, affine)])

Original shape: (96, 96, 19)
Resized to RAS shape: (100, 100, 40)


In [ ]:
nclass = 3
beta = 0.1

hmrf = TissueClassifierHMRF()
initial_segmentation, final_segmentation, PVE = hmrf.classify(ap, nclass, beta)

>> Iteration: 0
>> Iteration: 1
>> Iteration: 2
>> Iteration: 3
>> Iteration: 4
>> Iteration: 5
>> Iteration: 6
>> Iteration: 7
>> Iteration: 8
>> Iteration: 9
>> Iteration: 10
>> Iteration: 11
>> Iteration: 12
>> Iteration: 13
>> Iteration: 14
>> Iteration: 15
>> Iteration: 16
>> Iteration: 17
>> Iteration: 18
>> Iteration: 19
>> Iteration: 20
>> Iteration: 21
>> Iteration: 22
>> Iteration: 23


In [ ]:
new_image = (final_segmentation*80).astype(np.uint8)

In [ ]:
horizon(images=[(new_image, affine)])

Original shape: (96, 96, 19)
Resized to RAS shape: (100, 100, 40)
